# 필요 라이브러리

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import csv
import time
import re
from tqdm import tqdm

# 검색어 바탕 네이버 기사 url 수집

In [ ]:
def crawler(maxpage, query, s_date, e_date):
    naver_urls = []
    s_from = s_date.replace(".", "")
    e_to = e_date.replace(".", "")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1    
    
    while page < maxpage_t:
        
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + query + "&ds=" + s_date + "&de=" + e_date +  "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        # ua = UserAgent()
        # headers = {'User-Agent' : ua.random}

        req = requests.get(url)
        
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
        
        for urls in soup.select("a.info"):


            try:
                if "news.naver.com" in urls['href']:
                    naver_urls.append(urls['href'])
                  
            except Exception as e:
                continue
        page += 10
        
    return naver_urls
    
max_page = 400
query = '오피스 부동산'
s_date = '2019.01.01'
e_date = '2019.01.31'
year = ['2019','2020','2021']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
for j in list(range(0,3)):
    s_date = s_date.split(".")
    s_date[0] = year[j]
    s_date = ".".join(s_date)
    
    e_date = e_date.split(".")
    e_date[0] = year[j]
    e_date = ".".join(e_date)
    for i in list(range(0,12)):
        s_date = s_date.split(".")
        s_date[1] = month[i]
        s_date =  ".".join(s_date)

        
        e_date = e_date.split(".")
        e_date[1] = month[i]
        e_date =  ".".join(e_date)
        
        total_urls = crawler(max_page,query,s_date,e_date)
        
        
        total_urls = list(set(total_urls))
        
        # ConnectionError방지

        headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }


        titles = []
        contents=[]
        for i in tqdm(total_urls):
            original_html = requests.get(i,headers=headers)
            html = BeautifulSoup(original_html.text, "html.parser")
            # 검색결과확인시
            #print(html)

            #뉴스 제목 가져오기
            title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
            # list합치기
            title = ''.join(str(title))
            # html태그제거
            pattern1 = '<[^>]*>'
            title = re.sub(pattern=pattern1,repl='',string=title)
            titles.append(title)

            #뉴스 본문 가져오기

            content = html.select("div#dic_area")

            # 기사 텍스트만 가져오기
            # list합치기
            content = ''.join(str(content))

            #html태그제거 및 텍스트 다듬기
            content = re.sub(pattern=pattern1,repl='',string=content)
            pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
            content = content.replace(pattern2,'')

            contents.append(content)

        # 데이터프레임으로 정리(titles,url,contents)


        news_df = pd.DataFrame({'title': titles, 'link': total_urls, 'content': contents})
        news_df.to_csv(f"news_{query}from_{s_date}to_{e_date}.csv" ,index=False, encoding='utf-8-sig')
# # 검색어 및 날짜지정    
# maxpage = input("검색 할 페이지수: ")
# query = input("검색어: ")
# s_date = input("시작 날짜(YYYY.MM.DD): ")
# e_date = input("종료 날짜(YYYY.MM.DD): ")
# crawler(maxpage, query, s_date, e_date)
# print(len(naver_urls))
# print(len(list(set(naver_urls))))


 77%|█████████████████████████████████████████████████████████████▉                  | 179/231 [00:53<00:15,  3.36it/s]

In [5]:
s_date = '2019.01.01'
e_date = '2019.01.31'
year = ['2019','2020']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
for j in list(range(0,2)):
    s_date = s_date.split(".")
    s_date[0] = year[j]
    s_date = ".".join(s_date)
    
    e_date = e_date.split(".")
    e_date[0] = year[j]
    e_date = ".".join(e_date)
    for i in list(range(0,12)):


        s_date = s_date.split(".")
        s_date[1] = month[i]
        s_date =  ".".join(s_date)
        print(s_date)

        
        e_date = e_date.split(".")
        e_date[1] = month[i]
        e_date =  ".".join(e_date)
        print(e_date)
#         crawler(max_page,query,s_date,e_date)

2019.01.01
2019.01.31
2019.02.01
2019.02.31
2019.03.01
2019.03.31
2019.04.01
2019.04.31
2019.05.01
2019.05.31
2019.06.01
2019.06.31
2019.07.01
2019.07.31
2019.08.01
2019.08.31
2019.09.01
2019.09.31
2019.10.01
2019.10.31
2019.11.01
2019.11.31
2019.12.01
2019.12.31
2020.01.01
2020.01.31
2020.02.01
2020.02.31
2020.03.01
2020.03.31
2020.04.01
2020.04.31
2020.05.01
2020.05.31
2020.06.01
2020.06.31
2020.07.01
2020.07.31
2020.08.01
2020.08.31
2020.09.01
2020.09.31
2020.10.01
2020.10.31
2020.11.01
2020.11.31
2020.12.01
2020.12.31


# 해당 url의 제목, 본문 내용 수집

In [ ]:
# ConnectionError방지

headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }


titles = []
contents=[]
for i in tqdm(naver_urls):
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    #print(html)
    
    #뉴스 제목 가져오기
    title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1,repl='',string=title)
    titles.append(title)

    #뉴스 본문 가져오기

    content = html.select("div#dic_area")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))
    
    #html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1,repl='',string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2,'')

    contents.append(content)

# 데이터프레임으로 정리(titles,url,contents)


news_df = pd.DataFrame({'title': titles, 'link': naver_urls, 'content': contents})
news_df

# 뉴스 저장

In [ ]:
print(s_date)
news_df.to_csv(f"news_{query}from_{s_date}to_{e_date}.csv" ,index=False, encoding='utf-8-sig')

In [3]:
def crawler(maxpage, query, s_date, e_date):
    naver_urls = []
    s_from = s_date.replace(".", "")
    e_to = e_date.replace(".", "")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1    
    
    while page < maxpage_t:
        
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + query + "&ds=" + s_date + "&de=" + e_date +  "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        # ua = UserAgent()
        # headers = {'User-Agent' : ua.random}

        req = requests.get(url)
        
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
        
        for urls in soup.select("a.info"):


            try:
                if "news.naver.com" in urls['href']:
                    naver_urls.append(urls['href'])
                  
            except Exception as e:
                continue
        page += 10
        
    return naver_urls
# 검색어 및 날짜지정    
maxpage = input("검색 할 페이지수: ")
query = input("검색어: ")
s_date = input("시작 날짜(YYYY.MM.DD): ")
e_date = input("종료 날짜(YYYY.MM.DD): ")
total_urls = crawler(maxpage,query,s_date,e_date)


total_urls = list(set(total_urls))
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }


titles = []
contents=[]
for i in tqdm(total_urls):
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    #print(html)

    #뉴스 제목 가져오기
    title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1,repl='',string=title)
    titles.append(title)

    #뉴스 본문 가져오기

    content = html.select("div#dic_area")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    #html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1,repl='',string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2,'')

    contents.append(content)

# 데이터프레임으로 정리(titles,url,contents)


news_df = pd.DataFrame({'title': titles, 'link': total_urls, 'content': contents})
news_df.to_csv(f"news_{query}from_{s_date}to_{e_date}.csv" ,index=False, encoding='utf-8-sig')

검색 할 페이지수: 400
검색어: 리테일 부동산
시작 날짜(YYYY.MM.DD): 2019.01.01
종료 날짜(YYYY.MM.DD): 2019.01.31


100%|████████████████████████████████████████████████████████████████████████████████| 163/163 [00:46<00:00,  3.51it/s]
